## C6M3L3 Item 04 - Working with connection pools

## Prerequisites: 

To complete this lab, you need to install Python and MySQL. Then you need to install MySQL Connector/Python client or API on your Python environment. Follow the instructions in previous videos to install the required packages or software.  

To work with MySQL using Python, you must have an authorized user account on the MySQL server.  

## Scenario: 

Little Lemon’s guests need to access the database for any booking or inquiry, for example, reading the menu. Little Lemon, therefore, needs to establish a connection between the Python and MySQL databases for every operation.  Establishing a connection every time is resource intensive and it is affecting the performance of the Little Lemon application. To improve the performance of the application, Little Lemon needs to establish a pool of database connections to facilitate the guests’ inquiries to the database. 

You are tasked to help Little Lemon to create a pool of database connections using Python. 

## Task 1

Create a database connection pool with three connections available for the users to connect. You need to import MySQLConnectionPool class and pass the following arguments: 

* pool_name = “ll_pool_a” 
* pool_size = 3 
* **dbconfig 

Your database configuration will look like this:  
```Python
dbconfig = { 
    "database" : "name_of_the_little_lemon_database",  
    "user" : "your_username", 
    "password" : "your_password" 
} 
```

Use the actual name of the database together with authenticated username and password in the above configuration.  

Please use the Error class from `mysql.connector` to handle the possible error in case the wrong parameters are passed on the database configuration.  

**Tip:** Use try-except block from Python to implement the error handling. Once the connection pool is created, use the print statements to display the name of the pool and the number of connections in it.  

In [1]:
from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector import Error

dbconfig = {
    "database":"little_lemon",
    "user" : "root",
    "password" : ""
}

try:
    pool = MySQLConnectionPool(pool_name = "ll_pool_a",
                           pool_size = 3, #default is 5
                           **dbconfig)
    print("The connection pool is created with a name: ",pool.pool_name)
    print("The pool size is:",pool.pool_size)

except Error as er:
    print("Error code:", er.errno)
    print("Error message:", er.msg)

The connection pool is created with a name:  ll_pool_a
The pool size is: 3


## Task 2

Get a connection from the database connection pool that you have created in the first task and retrieve the following columns from the `Bookings` table: 

* `BookingID` 
* `GuestFirstName` 
* `GuestLastName` 

Retrieve the required columns and put the connection back into the pool after you have completed the task. 

**TIP:** Use the `get_connection` module from the `pool` to use the connection. Use `print` statements to display the following message and `close` the connection to return to the pool.  

In [2]:
from mysql.connector.pooling import MySQLConnectionPool

dbconfig = {
    "database":"little_lemon",
    "user" : "root",
    "password" : ""
}

pool = MySQLConnectionPool(pool_name = "ll_pool_a",
                           pool_size = 3, #default is 5
                           **dbconfig)


# Get the connection from the connection pool "pool"
print("Getting a connection from the pool.")
connection1 = pool.get_connection()

#print("A user with connection id {} is connected to the database.".format(
#    connection1.connection_id))

#db_Info = connection1.get_server_info()
#print("MySQL server version is:", db_Info)

# Create cursor object to communicate with entire MySQL database
print("Creating a cursor object.")
cursor = connection1.cursor()

# The SQL query is:
sql_query = "SELECT BookingId, GuestFirstName, GuestLastName FROM Bookings;"

# Execute query
print("Executing the SQL query.")
cursor.execute(sql_query)

# Fetch all results that satisfy the query 
print("Fetching the query results.")
results = cursor.fetchall()

# Retrieve column names
print("Retrieving the column names.")
cols = cursor.column_names

# Print column names and records in "results" using for loop
print("Printing the results.\n")
print("""Upcoming Bookings are:\n""")
print(cols)
for result in results:
    print(result)
    
# Put the connection back to the pool    
print("\nReturning the connection back to the pool.")
connection1.close()
print("The connection is placed back into the pool for the next user to connect.")

Getting a connection from the pool.
Creating a cursor object.
Executing the SQL query.
Fetching the query results.
Retrieving the column names.
Printing the results.

Upcoming Bookings are:

('BookingId', 'GuestFirstName', 'GuestLastName')
(1, 'Anna', 'Iversen')
(2, 'Joakim', 'Iversen')
(3, 'Vanessa', 'McCarthy')
(4, 'Marcos', 'Romero')
(5, 'Hiroki', 'Yamane')
(6, 'Diana', 'Pinto')

Returning the connection back to the pool.
The connection is placed back into the pool for the next user to connect.


## Task 3

The following five guests want to connect to the database: 

* guests = ["Anna", "Marcos", "Diana", "Joakim", "Hiroki"] 

You only have three connections in the database connection pool. Use the available connection in the `pool` to connect three guests and then add new connections in the pool to connect the remaining two guests. By adding more connection in the pool, make sure that all five guests are connected to the database at the same time.  

**Tip:** Use `add_connection` module from the `pool` and add a new connection if all are in use. Use `try-except` from Python and print the message to inform the user when connected. 

In [3]:
# Create a connection pool
from mysql.connector.pooling import MySQLConnectionPool
dbconfig = {
    "database":"little_lemon",
    "user" : "root",
    "password" : ""
}

pool = MySQLConnectionPool(pool_name = "ll_pool_a",
                           pool_size = 3, #default is 5
                           **dbconfig)

# List of the guests who want to connect to the database
guests = ["Anna", "Marcos", "Diana", "Joakim", "Hiroki"]

# To add connection to the pool, the connection must be of MySQLConnection instance 
# Also possible to create via connect module and need the import below
import mysql.connector as connector

# Assign connection to each user
for guest in guests:
    try:
        guest_connected = pool.get_connection()
        print("[{}] is connected.\n".format(guest))
    except:
        print("No more connections are available.")
        print("Adding new connection in the pool.")
        
        # Create a connection
        connection=connector.connect(user="root",password="")
        # Add the connection into the pool
        pool.add_connection(cnx=connection)
        print("A new connection is added in the pool.\n")
        
        user_connected = pool.get_connection()
        print("[{}] is connected.\n".format(guest))

[Anna] is connected.

[Marcos] is connected.

[Diana] is connected.

No more connections are available.
Adding new connection in the pool.
A new connection is added in the pool.

[Joakim] is connected.

No more connections are available.
Adding new connection in the pool.
A new connection is added in the pool.

[Hiroki] is connected.

